<a href="https://colab.research.google.com/github/manapole/TA/blob/main/dataset_alice_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAREA 3.1
##Creación de un dataset en HuggingFace

Alumna: María Cira Napoletano

Enlace al modelo: https://huggingface.co/manapole/alice-mt


In [ ]:
!pip install datasets evaluate transformers[sentencepiece] sacrebleu accelerate -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("manapole/alice_es_en")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/315 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['spanish text', 'english text'],
        num_rows: 360
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['spanish text', 'english text'],
    num_rows: 360
})

In [ ]:
dataset['train'].to_pandas()

,spanish text,english text
0,LAS AVENTURAS DE ALICIA EN EL PAÍS DE LAS MARA...,ALICE’S ADVENTURES IN WONDERLAND
1,CAPÍTULO I. Por la madriguera del conejo,CHAPTER I. Down the Rabbit-Hole
2,Alicia empezaba a estar muy cansada de estar s...,Alice was beginning to get very tired of sitti...
3,dos veces había mirado el libro que su hermana...,twice she had peeped into the book her sister ...
4,"“¿y de qué sirve un libro,” pensó Alicia, “sin...","‘and what is the use of a book,’ thought Alice..."
...,...,...
355,"“¿y de qué sirve un libro,” pensó Alicia, “sin...","‘and what is the use of a book,’ thought Alice..."
356,Así que estaba considerando en su propia mente...,So she was considering in her own mind (as wel...
357,"torpe), si valdría la pena el placer de hacer ...","stupid), whether the pleasure of making a dais..."
358,"margaritas, cuando de repente un Conejo Blanco...","daisies, when suddenly a White Rabbit with pin..."


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
max_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["spanish text"],
        max_length=max_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["english text"], max_length=max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['spanish text', 'english text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 360
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer)

# MODELO

In [ ]:
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    # Carpeta donde se guardará el modelo
    output_dir="alice-mt",
    # Evaluar el modelo cada época
    eval_strategy="no",
    # Valores para modificar los pesos del modelo
    learning_rate=5.6e-5,
    weight_decay=0.01,
    # Número de ejemplos que se muestran cada vez al modelo
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Número de veces que que se muestran todos los datos al modelo
    num_train_epochs=2,
    # Realizar predicciones al validar
    predict_with_generate=True,
    push_to_hub=True,
    report_to="none"
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
import numpy as np

import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["test"], # Removed as there is no 'test' split
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-28-914583488.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=90, training_loss=1.2283515082465277, metrics={'train_runtime': 454.019, 'train_samples_per_second': 1.586, 'train_steps_per_second': 0.198, 'total_flos': 48218637336576.0, 'train_loss': 1.2283515082465277, 'epoch': 2.0})

# COMPARTIENDO Y USANDO EL MODELO

In [ ]:
%cd alice-mt
trainer.push_to_hub(commit_message="Training complete", tags="simplification")

/content/alice-mt


CommitInfo(commit_url='https://huggingface.co/manapole/alice-mt/commit/c4ae9f8af05481d3a9d27cee1252698373f334a3', commit_message='Training complete', commit_description='', oid='c4ae9f8af05481d3a9d27cee1252698373f334a3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/manapole/alice-mt', endpoint='https://huggingface.co', repo_type='model', repo_id='manapole/alice-mt'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline
neutralizer = pipeline('text2text-generation', model='manapole/alice-mt')

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

Device set to use cuda:0
